In [1]:
import argparse
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import data_loader.preprocess as module_preprocess
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer


# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [2]:
import json
config = ConfigParser(json.load(open('config.json', 'r')))

In [3]:
logger = config.get_logger('train')
# preprocess
# config.init_obj('preprocess', module_preprocess)

# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data)
valid_data_loader = data_loader.split_validation()

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release 

In [25]:
from torchtext.data import Iterator

In [26]:
train_iter = Iterator(data_loader.dataset, 32)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [35]:
val_iter = Iterator(valid_data_loader.dataset, 32)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [36]:
for idx, i in enumerate(val_iter):
    if idx == 5:
        break
    print(i.talk)
    print(i.response)


tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   7,    7,  338,  ...,  691,    7,   97],
        [  54, 1275,   25,  ...,    5,   50,   15],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  7,  60,  13,  ...,  58,   6,  52],
        [ 54,  54,  34,  ..., 127, 422,  14],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])
tensor([[    2,     2,     2,  ...,     2,     2,     2],
        [    7,    86,     6,  ...,    28,  4787,    39],
        [ 1402,     4,    17,  ..., 12003,     5,     4],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([[   2,    2,  

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
